In [1]:
!python --version

Python 3.7.4


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

/home/aspa2/rockmelon/MMXXII/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/aspa2/rockmelon/MMXXII/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/aspa2/rockmelon/MMXXII/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/aspa2/rockmelon/MMXXII/lib/python3.7/site-packages/tensorf

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14927021366967009103
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 8980648658305112134
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 8568150889765299761
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11339952948
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11369349136155941867
physical_device_desc: "device: 0, name: TITAN X (Pascal), pci bus id: 0000:01:00.0, compute capability: 6.1"
]


/home/aspa2/rockmelon/MMXXII/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/aspa2/rockmelon/MMXXII/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/aspa2/rockmelon/MMXXII/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/aspa2/rockmelon/MMXXII/lib/python3.7/

In [3]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  1


2022-01-22 17:20:55.660331: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-22 17:20:55.660989: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with properties: 
name: TITAN X (Pascal) major: 6 minor: 1 memoryClockRate(GHz): 1.531
pciBusID: 0000:01:00.0
2022-01-22 17:20:55.661037: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcudart.so.10.0
2022-01-22 17:20:55.661050: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcublas.so.10.0
2022-01-22 17:20:55.661059: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcufft.so.10.0
2022-01-22 17:20:55.661069: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libc

In [4]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2

from mrcnn.visualize import display_instances
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = r"/home/aspa2/rockmelon/MMXXII"

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils

# Path to trained weights file
COCO_WEIGHTS_PATH = r"/home/aspa2/rockmelon/MMXXII/mask_rcnn_coco.h5"

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
#DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")
DEFAULT_LOGS_DIR = r"/media/aspa2/KINGSTON/SummerResearch/MMXXII/logs"


class CustomConfig(Config):
    """Configuration for training on the custom  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "object"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + rockmelon

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 10

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9
    
    # Others
    #BACKBONE = "resnet50"
    BATCH_SIZE = 23
    TRAIN_ROIS_PER_IMAGE = 512
    #WEIGHT_DECAY = 0.0001

############################################################
#  Dataset
############################################################

class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir, subset):
        """Load a subset of the Dog-Cat dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have only one class to add.
        self.add_class("object", 1, "rockmelon")

        # Train or validation dataset?
        # assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, "train")

        # We mostly care about the x and y coordinates of each region
        annotations1 = json.load(open(r'/home/aspa2/rockmelon/MMXXII/Dataset/train/train_json.json'))
        #annotations1 = json.load(open(os.path.join(dataset_dir, "%s_json.json"%(subset))))
        # print(annotations1)
        annotations = list(annotations1.values())  # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]
        
        # Add images
        for a in annotations:
            # print(a)
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. There are stores in the
            # shape_attributes (see json format above)
            polygons = [r['shape_attributes'] for r in a['regions']] 
            objects = [s['region_attributes']['name'] for s in a['regions']]
            print("objects:",objects)
            name_dict = {"rockmelon": 1}

            # key = tuple(name_dict)
            num_ids = [name_dict[a] for a in objects]
     
            # num_ids = [int(n['Event']) for n in objects]
            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            print("numids",num_ids)
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "object",  ## for a single class just add the name here
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
                )

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a Dog-Cat dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
        	rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

        	mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        # Map class names to class IDs.
        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids #np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

def train(model):
    """Train the model."""
    # Training dataset.
    dataset_train = CustomDataset()
    dataset_train.load_custom(r"/home/aspa2/rockmelon/MMXXII/Dataset", "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = CustomDataset()
    dataset_val.load_custom(r"/home/aspa2/rockmelon/MMXXII/Dataset", "val")
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=3000,
                layers='heads')
			
				
				
config = CustomConfig()
model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=DEFAULT_LOGS_DIR)

weights_path = COCO_WEIGHTS_PATH
        # Download weights file
if not os.path.exists(weights_path):
  utils.download_trained_weights(weights_path)

model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])

history = train(model)

Using TensorFlow backend.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead


2022-01-22 17:21:15.755115: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-22 17:21:15.756116: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with properties: 
name: TITAN X (Pascal) major: 6 minor: 1 memoryClockRate(GHz): 1.531
pciBusID: 0000:01:00.0
2022-01-22 17:21:15.756155: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcudart.so.10.0
2022-01-22 17:21:15.756166: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcublas.so.10.0
2022-01-22 17:21:15.756175: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcufft.so.10.0
2022-01-22 17:21:15.756184: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libc

objects: ['rockmelon', 'rockmelon', 'rockmelon']
numids [1, 1, 1]
objects: ['rockmelon', 'rockmelon']
numids [1, 1]
objects: ['rockmelon', 'rockmelon']
numids [1, 1]
objects: ['rockmelon', 'rockmelon', 'rockmelon', 'rockmelon', 'rockmelon']
numids [1, 1, 1, 1, 1]
objects: ['rockmelon', 'rockmelon', 'rockmelon', 'rockmelon', 'rockmelon', 'rockmelon']
numids [1, 1, 1, 1, 1, 1]
objects: ['rockmelon', 'rockmelon', 'rockmelon', 'rockmelon']
numids [1, 1, 1, 1]
objects: ['rockmelon', 'rockmelon', 'rockmelon', 'rockmelon', 'rockmelon', 'rockmelon']
numids [1, 1, 1, 1, 1, 1]
objects: ['rockmelon', 'rockmelon']
numids [1, 1]
objects: ['rockmelon']
numids [1]
objects: ['rockmelon', 'rockmelon', 'rockmelon', 'rockmelon', 'rockmelon', 'rockmelon']
numids [1, 1, 1, 1, 1, 1]
objects: ['rockmelon', 'rockmelon', 'rockmelon', 'rockmelon']
numids [1, 1, 1, 1]
objects: ['rockmelon', 'rockmelon', 'rockmelon']
numids [1, 1, 1]
objects: ['rockmelon']
numids [1]
objects: ['rockmelon', 'rockmelon', 'rockmelon

objects: ['rockmelon', 'rockmelon', 'rockmelon', 'rockmelon', 'rockmelon']
numids [1, 1, 1, 1, 1]
objects: ['rockmelon', 'rockmelon', 'rockmelon']
numids [1, 1, 1]
objects: ['rockmelon', 'rockmelon', 'rockmelon', 'rockmelon', 'rockmelon']
numids [1, 1, 1, 1, 1]
objects: ['rockmelon', 'rockmelon', 'rockmelon']
numids [1, 1, 1]
objects: ['rockmelon', 'rockmelon']
numids [1, 1]
objects: ['rockmelon', 'rockmelon', 'rockmelon']
numids [1, 1, 1]
objects: ['rockmelon']
numids [1]
objects: ['rockmelon', 'rockmelon']
numids [1, 1]
objects: ['rockmelon', 'rockmelon']
numids [1, 1]
objects: ['rockmelon', 'rockmelon', 'rockmelon']
numids [1, 1, 1]
objects: ['rockmelon', 'rockmelon', 'rockmelon', 'rockmelon']
numids [1, 1, 1, 1]
objects: ['rockmelon', 'rockmelon']
numids [1, 1]
objects: ['rockmelon']
numids [1]
objects: ['rockmelon', 'rockmelon', 'rockmelon']
numids [1, 1, 1]
objects: ['rockmelon', 'rockmelon', 'rockmelon', 'rockmelon', 'rockmelon', 'rockmelon', 'rockmelon', 'rockmelon', 'rockmelon

...

10/10 [==============================] - 28s 3s/step - loss: 0.0453 - rpn_class_loss: 1.9688e-04 - rpn_bbox_loss: 0.0020 - mrcnn_class_loss: 0.0043 - mrcnn_bbox_loss: 0.0021 - mrcnn_mask_loss: 0.0366 - val_loss: 0.0434 - val_rpn_class_loss: 4.0698e-04 - val_rpn_bbox_loss: 0.0023 - val_mrcnn_class_loss: 0.0035 - val_mrcnn_bbox_loss: 0.0020 - val_mrcnn_mask_loss: 0.0352
Epoch 2900/3000
10/10 [==============================] - 28s 3s/step - loss: 0.0448 - rpn_class_loss: 1.1214e-04 - rpn_bbox_loss: 0.0017 - mrcnn_class_loss: 0.0045 - mrcnn_bbox_loss: 0.0025 - mrcnn_mask_loss: 0.0360 - val_loss: 0.0444 - val_rpn_class_loss: 2.4483e-04 - val_rpn_bbox_loss: 0.0020 - val_mrcnn_class_loss: 0.0047 - val_mrcnn_bbox_loss: 0.0022 - val_mrcnn_mask_loss: 0.0351
Epoch 2901/3000
10/10 [==============================] - 28s 3s/step - loss: 0.0377 - rpn_class_loss: 6.7342e-04 - rpn_bbox_loss: 0.0013 - mrcnn_class_loss: 0.0019 - mrcnn_bbox_loss: 0.0018 - mrcnn_mask_loss: 0.0321 - val_loss: 0.0453 - val_r

10/10 [==============================] - 28s 3s/step - loss: 0.0434 - rpn_class_loss: 3.8473e-04 - rpn_bbox_loss: 0.0020 - mrcnn_class_loss: 0.0038 - mrcnn_bbox_loss: 0.0019 - mrcnn_mask_loss: 0.0354 - val_loss: 0.0420 - val_rpn_class_loss: 3.7286e-04 - val_rpn_bbox_loss: 0.0018 - val_mrcnn_class_loss: 0.0031 - val_mrcnn_bbox_loss: 0.0020 - val_mrcnn_mask_loss: 0.0348
Epoch 2921/3000
10/10 [==============================] - 28s 3s/step - loss: 0.0389 - rpn_class_loss: 1.8872e-04 - rpn_bbox_loss: 0.0016 - mrcnn_class_loss: 0.0030 - mrcnn_bbox_loss: 0.0019 - mrcnn_mask_loss: 0.0323 - val_loss: 0.0444 - val_rpn_class_loss: 3.7192e-04 - val_rpn_bbox_loss: 0.0034 - val_mrcnn_class_loss: 0.0026 - val_mrcnn_bbox_loss: 0.0018 - val_mrcnn_mask_loss: 0.0362
Epoch 2922/3000
10/10 [==============================] - 28s 3s/step - loss: 0.0400 - rpn_class_loss: 8.2247e-05 - rpn_bbox_loss: 0.0018 - mrcnn_class_loss: 0.0030 - mrcnn_bbox_loss: 0.0018 - mrcnn_mask_loss: 0.0334 - val_loss: 0.0396 - val_r

10/10 [==============================] - 28s 3s/step - loss: 0.0531 - rpn_class_loss: 0.0021 - rpn_bbox_loss: 0.0037 - mrcnn_class_loss: 0.0031 - mrcnn_bbox_loss: 0.0026 - mrcnn_mask_loss: 0.0416 - val_loss: 0.0480 - val_rpn_class_loss: 7.8795e-04 - val_rpn_bbox_loss: 0.0034 - val_mrcnn_class_loss: 0.0030 - val_mrcnn_bbox_loss: 0.0024 - val_mrcnn_mask_loss: 0.0385
Epoch 2942/3000
10/10 [==============================] - 28s 3s/step - loss: 0.0461 - rpn_class_loss: 4.7408e-04 - rpn_bbox_loss: 0.0023 - mrcnn_class_loss: 0.0040 - mrcnn_bbox_loss: 0.0020 - mrcnn_mask_loss: 0.0374 - val_loss: 0.0436 - val_rpn_class_loss: 9.0999e-04 - val_rpn_bbox_loss: 0.0018 - val_mrcnn_class_loss: 0.0033 - val_mrcnn_bbox_loss: 0.0021 - val_mrcnn_mask_loss: 0.0355
Epoch 2943/3000
10/10 [==============================] - 28s 3s/step - loss: 0.0463 - rpn_class_loss: 3.5083e-04 - rpn_bbox_loss: 0.0020 - mrcnn_class_loss: 0.0048 - mrcnn_bbox_loss: 0.0024 - mrcnn_mask_loss: 0.0367 - val_loss: 0.0409 - val_rpn_c

10/10 [==============================] - 28s 3s/step - loss: 0.0570 - rpn_class_loss: 3.9454e-04 - rpn_bbox_loss: 0.0030 - mrcnn_class_loss: 0.0059 - mrcnn_bbox_loss: 0.0027 - mrcnn_mask_loss: 0.0450 - val_loss: 0.0550 - val_rpn_class_loss: 1.9990e-04 - val_rpn_bbox_loss: 0.0025 - val_mrcnn_class_loss: 0.0037 - val_mrcnn_bbox_loss: 0.0031 - val_mrcnn_mask_loss: 0.0456
Epoch 2963/3000
10/10 [==============================] - 28s 3s/step - loss: 0.0589 - rpn_class_loss: 4.0112e-04 - rpn_bbox_loss: 0.0034 - mrcnn_class_loss: 0.0038 - mrcnn_bbox_loss: 0.0030 - mrcnn_mask_loss: 0.0482 - val_loss: 0.0507 - val_rpn_class_loss: 3.3154e-04 - val_rpn_bbox_loss: 0.0030 - val_mrcnn_class_loss: 0.0037 - val_mrcnn_bbox_loss: 0.0026 - val_mrcnn_mask_loss: 0.0410
Epoch 2964/3000
10/10 [==============================] - 28s 3s/step - loss: 0.0674 - rpn_class_loss: 1.8241e-04 - rpn_bbox_loss: 0.0069 - mrcnn_class_loss: 0.0045 - mrcnn_bbox_loss: 0.0032 - mrcnn_mask_loss: 0.0525 - val_loss: 0.0558 - val_r

10/10 [==============================] - 28s 3s/step - loss: 0.0446 - rpn_class_loss: 4.2718e-04 - rpn_bbox_loss: 0.0021 - mrcnn_class_loss: 0.0051 - mrcnn_bbox_loss: 0.0020 - mrcnn_mask_loss: 0.0350 - val_loss: 0.0427 - val_rpn_class_loss: 7.4236e-04 - val_rpn_bbox_loss: 0.0018 - val_mrcnn_class_loss: 0.0030 - val_mrcnn_bbox_loss: 0.0021 - val_mrcnn_mask_loss: 0.0350
Epoch 2984/3000
10/10 [==============================] - 28s 3s/step - loss: 0.0476 - rpn_class_loss: 2.1510e-04 - rpn_bbox_loss: 0.0021 - mrcnn_class_loss: 0.0064 - mrcnn_bbox_loss: 0.0021 - mrcnn_mask_loss: 0.0367 - val_loss: 0.0429 - val_rpn_class_loss: 7.0256e-04 - val_rpn_bbox_loss: 0.0018 - val_mrcnn_class_loss: 0.0037 - val_mrcnn_bbox_loss: 0.0019 - val_mrcnn_mask_loss: 0.0348
Epoch 2985/3000
10/10 [==============================] - 28s 3s/step - loss: 0.0406 - rpn_class_loss: 9.9808e-05 - rpn_bbox_loss: 0.0015 - mrcnn_class_loss: 0.0026 - mrcnn_bbox_loss: 0.0019 - mrcnn_mask_loss: 0.0345 - val_loss: 0.0423 - val_r

In [6]:
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.title('metrics')
plt.ylabel('value')
plt.xlabel('epoch no.')
plt.legend(loc='upper right')
plt.show()

None


AttributeError: 'NoneType' object has no attribute 'history'